In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


local
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved.cu -o /mnt/d//packing/temp/pack_cuda_crystal.cubin
ptxas info    : 0 bytes gmem, 1172 bytes cmem[3]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 36 registers, used 1 barriers, 404 bytes cmem[0], 16 bytes cmem[2]
ptxas info    : Compile time = 7.726 ms
ptxas info    : Compiling entry function 'multi_boundary_list_total' for 'sm_89'
ptxas info  

In [ ]:
%load_ext pyinstrument

In [ ]:
%%pyinstrument
kgs.profiling=True
# %matplotlib widget
kgs.enable_module_profiling(kgs)
import pack_ga3
import pack_runner
fastMode = False
kgs.debugging_mode = 2

runner = pack_ga3.baseline()
runner.ga.ga_base.N_trees_to_do = 156
ga.legalize=False
ga.ga_base.legalize=False
runner.n_generations = 3
runner.diagnostic_plot = False
runner.seed = 0

runner.run()

[ 10  11  13  15  17  20  23  27  31  36  42  49  56  65  76  87 101 117
 135 156 181 209 242 280 323 374 432 500]
[np.int64(10), np.int64(11), np.int64(13), np.int64(15), np.int64(17), np.int64(20), np.int64(23), np.int64(27), np.int64(31), np.int64(36), np.int64(42), np.int64(49), np.int64(56), np.int64(65), np.int64(76), np.int64(87), np.int64(101), np.int64(117), np.int64(135), np.int64(156), np.int64(181), np.int64(209), np.int64(242), np.int64(280), np.int64(323), np.int64(374), np.int64(432), np.int64(500)]
[1, 2, 3, 4, 5, 6, 7, 8, 9, np.int64(10), np.int64(11), np.int64(13), np.int64(15), np.int64(17), np.int64(20), np.int64(23), np.int64(27), np.int64(31), np.int64(36), np.int64(42), np.int64(49), np.int64(56), np.int64(65), np.int64(76), np.int64(87), np.int64(101), np.int64(117), np.int64(135), np.int64(156), np.int64(181), np.int64(209), np.int64(242), np.int64(280), np.int64(323), np.int64(374), np.int64(432), np.int64(500)]
[ 10  11  13  15  17  20  23  27  31  36  42  49

/mnt/d/packing/code/analysis/../core/pack_dynamics.py:60: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  x0 = from_dlpack(x0.toDlpack())
/mnt/d/packing/code/analysis/../core/pack_dynamics.py:92: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  return from_dlpack(tmp_cost[:N].toDlpack()), from_dlpack(res.toDlpack())


Generation 0: Best costs = [[0.339005, 1.958348]]
